In [1]:
%pip install -r requirements.txt
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import cv2

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ejfxt100
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ejfxt100
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Load the pretrained model
model = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')
# Set model to evaluation mode
model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
def get_vector(img):
    img = transform(img)
    img = img[:3]
    img = normalize(img)
    img = img.unsqueeze(0)
    print(img.shape)
    # Initialize the embedding tensor
    my_embedding = torch.zeros(512, dtype=torch.float32)

    # Hook to extract the features from the avgpool layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.flatten())  # Flatten the output

    hook = layer.register_forward_hook(copy_data)

    # Forward pass to compute the embedding
    with torch.no_grad():
        model(img)

    hook.remove()  # Remove the hook
    return my_embedding

In [5]:
files = ['cat1.png', 'cat2.png', 'dog1.png', 'dog2.png']
vectors = []
for file in files:
    img = Image.open(file)
    vec = get_vector(img)
    vectors.append(vec)

def similarity(v1, v2):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    cos_sim = cos(v1.unsqueeze(0),
                v2.unsqueeze(0))
    return cos_sim

# Display a matrix of cosine similarity scores
sims = np.zeros((len(files), len(files)))
for i in range(len(files)):
    for j in range(len(files)):
        sims[i, j] = similarity(vectors[i], vectors[j])

print("Similarity Matrix:")
print("\t" + "\t".join([file.split(".")[0] for file in files]))
for i in range(len(files)):
    print(files[i].split(".")[0] + "\t" + "\t".join([f"{s:.2f}" for s in sims[i]]))

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
Similarity Matrix:
	cat1	cat2	dog1	dog2
cat1	1.00	0.80	0.57	0.54
cat2	0.80	1.00	0.45	0.52
dog1	0.57	0.45	1.00	0.54
dog2	0.54	0.52	0.54	1.00
